In [1]:
import os
from tbp.monty.frameworks.utils.logging_utils import load_stats

import matplotlib.pyplot as plt
from tbp.monty.frameworks.utils.plot_utils import plot_graph
import numpy as np

In [2]:
%matplotlib widget

In [3]:
pretrain_path = os.path.expanduser("~/tbp/results/monty/pretrained_models/")
pretrained_dict = pretrain_path + "pretrained_ycb_v10/surf_agent_1lm_tbp_robot_lab/pretrained/"

log_path = os.path.expanduser("~/tbp/results/monty/projects/evidence_eval_runs/logs/")
exp_name = "json_dataset_ultrasound_learning"
exp_path = log_path + exp_name

In [ ]:
train_stats, eval_stats, detailed_stats, lm_models = load_stats(exp_path,
                                                                load_train=False, # doesn't load train csv
                                                                load_eval=False, # loads eval_stats.csv
                                                                load_detailed=False, # doesn't load .json
                                                                load_models=True, # loads .pt models
                                                                pretrained_dict=pretrained_dict,
                                                               )

In [ ]:
model = lm_models['0']['LM_0']['new_object0']['patch']
plot_graph(model, rotation=120)
plt.show()

In [ ]:
train_stats, eval_stats, detailed_stats, lm_models = load_stats(exp_path,
                                                                load_train=False, # doesn't load train csv
                                                                load_eval=False, # loads eval_stats.csv
                                                                load_detailed=False, # doesn't load .json
                                                                load_models=True, # loads .pt models
                                                                pretrained_dict=pretrained_dict,
                                                               )
model = lm_models['0']['LM_0']['new_object0']['patch']
locs = np.array(model.pos)
normals = np.array(model.norm)

In [110]:
not_artifacts = np.where(locs[:,2] > -0.7)
locs = locs[not_artifacts]
normals = normals[not_artifacts]

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1, projection="3d")
# color points by id in array
colors = np.linspace(0, 1, len(locs))
ax.scatter(locs[:,0], locs[:,1], locs[:,2], c=colors)
# add point normals to plot
ax.quiver(locs[:,0], locs[:,1], locs[:,2], normals[:,0], normals[:,1], normals[:,2], length=0.01, color="red")

ax.set_aspect("equal")
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("z")
fig.tight_layout()
plt.show()


In [ ]:
lm_models['pretrained'][0].keys()

In [ ]:
model = lm_models['pretrained'][0]['potted_meat_can']['patch']
locs = np.array(model.pos)
normals = np.array(model.norm)
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1, projection="3d")
# color points by id in array
colors = np.linspace(0, 1, len(locs))
ax.scatter(locs[:,0], locs[:,1], locs[:,2], c=colors)
# add point normals to plot
ax.quiver(locs[:,0], locs[:,1], locs[:,2], normals[:,0], normals[:,1], normals[:,2], length=0.005, color="red")

ax.set_aspect("equal")
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("z")
fig.tight_layout()
plt.show()

In [40]:
tracker_locations = np.load("tracker_locations.npy")
tracker_orientations = np.load("tracker_orientations.npy")
depths = np.load("depths.npy")
fake_normal_rel_world = np.load("fake_normal_rel_world.npy")
normal_rel_world = np.load("normal_rel_world.npy")

In [56]:
import quaternion as qt
def get_patch_world_location_alt(
    agent_position,
    sensor_position,
    agent_rotation,
    image_depth,
):
    """Calculates the patch's location and orientation in world coordinates."""
    # Move the agent position by sensor_position[1] in the direction of the agent's local Y axis.
    # This is because the sensor is offset from the agent's center by sensor_position[1] in the
    # local Y axis direction.
    offset_direction = np.array([0.0, 0.0, 1.0])
    rotated_offset_direction = (
        agent_rotation @ offset_direction
    )
    offset_distance = sensor_position[1]  # = 0.03
    relative_offset = offset_distance * rotated_offset_direction
    agent_position = agent_position + relative_offset
    fake_sensor_rel_world = agent_rotation @ np.array(
        [0.0, 1.0, 0.0]
    )
    # TODO: see how to incorporate the sensor offset in y direction
    offset_distance = sensor_position[2] + image_depth

    # Determine the movement vector.
    # The vector is in the opposite direction of the unit_normal (-unit_normal)
    # and scaled by the offset_distance.
    movement_vector = -fake_sensor_rel_world * offset_distance

    # Compute the new location by applying the movement_vector to the agent's current position.
    patch_world_location = agent_position + movement_vector

    return patch_world_location

In [ ]:
world_locations = []
for i in range(len(tracker_locations)):
    world_locations.append(get_patch_world_location_alt(
        tracker_locations[i],
        [0.0, 0.03, 0.09],
        tracker_orientations[i],
        depths[i],
    ))
world_locations = np.array(world_locations)
world_locations.shape


In [ ]:

locs = np.array(world_locations)
normals = np.array(normal_rel_world)
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1, projection="3d")
# color points by id in array
colors = np.linspace(0, 1, len(locs))
ax.scatter(locs[:,0], locs[:,1], locs[:,2], c=colors)
# add point normals to plot
ax.quiver(locs[:,0], locs[:,1], locs[:,2], normals[:,0], normals[:,1], normals[:,2], length=0.005, color="red")

ax.set_aspect("equal")
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("z")
fig.tight_layout()
plt.show()

In [29]:
logging_data = np.load('logging_data.npz')

In [ ]:
logging_data.files

In [ ]:
logging_data['tracker_position']